In [85]:
import time
def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Time taken to execute {func.__name__}: {elapsed_time:.4f} seconds")
        return result
    return wrapper

In [86]:
!pip install pandas
!pip install pyarrow
pyarrow --version

NameError: name 'pyarrow' is not defined

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa

print("PyArrow version:", pa.__version__)

file_path = "traffic_data/traffic_data_sample.parquet"
output_file_path = "cleaned_data/traffic_data_sample_filtered.parquet"
output_csv_file_path = "cleaned_data/traffic_data_sample_filtered.csv"


table = pq.read_table(file_path)
df = table.to_pandas()

# Drop the columns 'day_type' and 'day_part'

print(df.columns)
df['highway'] = df['highway'].str.lower()

print("Unique values in 'highway' column before filtering:")
print(df['highway'].unique())
print(df['year'].unique())
print(df['month'].unique())

# remove states other than GA
df_filtered = df[df['state_code'] == 'GA']
print(df_filtered['state_code'].value_counts())

columns_to_drop = ["created_at","updated_at", "trips_volume_masked","trips_sample_count_masked", "segment_name", "geom","match_dir", "county_fips", 'day_type', 'day_part', 'mode', "segment_id", "state_code", "country_code", "vmt", "year", "month"]
df_filtered.drop(columns=columns_to_drop, inplace=True)
df_filtered = df_filtered[~df_filtered.drop(columns=['id']).duplicated()]

# Print unique values in the 'highway' column after filtering
print("Unique values in 'highway' column after filtering:")
print(df_filtered['highway'].unique())
print(df_filtered.columns)


@measure_time
def groupby_count():
    return df_filtered.groupby('highway').size()

groupby_count()

PyArrow version: 17.0.0
Index(['id', 'mode', 'country_code', 'state_code', 'county_fips',
       'segment_name', 'osm_id', 'highway', 'segment_length_m', 'segment_id',
       'year', 'month', 'match_dir', 'day_type', 'day_part', 'trips_volume',
       'trips_sample_count', 'vmt', 'geom', 'created_at', 'updated_at',
       'trips_volume_masked', 'trips_sample_count_masked'],
      dtype='object')
Unique values in 'highway' column before filtering:
['primary' 'secondary' 'motorway' 'tertiary' 'unclassified' 'trunk'
 'motorway_link' 'trunk_link' 'primary_link' 'tertiary_link'
 'secondary_link' 'living_street' 'track' 'cycleway' 'footway' 'bridleway'
 'path' 'pedestrian']
[2023]
[3]
state_code
GA    675498
Name: count, dtype: int64
Unique values in 'highway' column after filtering:
['primary' 'secondary' 'motorway' 'tertiary' 'unclassified' 'trunk'
 'motorway_link' 'trunk_link' 'primary_link' 'tertiary_link'
 'secondary_link' 'living_street' 'track' 'cycleway' 'footway']
Index(['id', 'osm_

/var/folders/kq/cs03j9553ng9l_hc6_yh2qyw0000gn/T/ipykernel_91647/4164563080.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=columns_to_drop, inplace=True)


highway
cycleway               6
footway               17
living_street        216
motorway            6341
motorway_link       7385
primary            87452
primary_link        3370
secondary         150389
secondary_link      2045
tertiary          214973
tertiary_link        993
track                 26
trunk              46547
trunk_link          4586
unclassified       65439
dtype: int64

In [ ]:
df = df_filtered
df.to_parquet(output_file_path, index=False)
df.to_csv(output_csv_file_path, index=False)

In [ ]:
# Ensure the specific_osm_id is of the same type as the osm_id column
specific_osm_id = 1136684141  # Replace with the osm_id you are looking for

# Print the first few rows of the DataFrame to check the structure
# print("DataFrame structure:")
# print(df.head())

# Check the data type of the osm_id column
print(f"osm_id column data type: {df['osm_id'].dtype}")

# If the osm_id column is of type str, convert specific_osm_id to str
if df['osm_id'].dtype == 'object':
    specific_osm_id = str(specific_osm_id)

# Check if the specific_osm_id exists in the osm_id column
if specific_osm_id in df['osm_id'].values:
    row = df[df['osm_id'] == specific_osm_id]
    print("Row found:")
    print(row)
else:
    print(f"osm_id {specific_osm_id} not found in the DataFrame")

osm_id column data type: int64
Row found:
             id      osm_id   highway  segment_length_m  trips_volume  \
182497  2952579  1136684141  motorway           656.998         72963   

        trips_sample_count  
182497              741221  


In [ ]:
import requests

def get_address_from_osm_id(osm_ids):
    base_url = "https://nominatim.openstreetmap.org/lookup"
    params = {
        "osm_ids": ",".join(map(str, osm_ids)),  # List of OSM IDs formatted correctly
        "format": "json",  # Request JSON response
    }
    headers = {
        "User-Agent": "YourApp/1.0 (aaryaman04@gmail.com)"  # Required to avoid being blocked
    }

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
# List of OSM IDs to look up from the DataFrame
osm_ids = df['osm_id'].unique()[:10]
print(f"OSM IDs to look up: {osm_ids}")

result = get_address_from_osm_id(osm_ids)

if result:
    for place in result:
        print(f"OSM ID: {place['osm_id']}, Address: {place.get('display_name', 'No address found')}")
else:
    print("No results returned")


OSM IDs to look up: [1075671961    9001576   56444638    9172029    9172224  324271283
  805252387   83442395 1053530914    9384326]
No results returned


In [ ]:
# from geopy.geocoders import Nominatim
# import requests

# def get_address_from_osm_id(osm_id):
#     """
#     Get the address of a road segment given its OSM ID.
#     """
#     nominatim_url = f"https://nominatim.openstreetmap.org/reverse?osm_type=W&osm_id={osm_id}&format=json"
    
#     print(f"Request URL: {nominatim_url}")  # Debugging: Print the request URL
    
#     response = requests.get(nominatim_url, headers={"User-Agent": "OSM-Traffic-Analysis"})
    
#     if response.status_code == 200:
#         data = response.json()
#         return data.get("display_name", "Address not found")
#     else:
#         print(f"Error: Received status code {response.status_code}")
#         print(f"Response content: {response.text}")  # Debugging: Print the response content
#         return None

# # Example usage
# osm_id = 123456789  # Replace with actual OSM ID
# address = get_address_from_osm_id(osm_id)
# print("Address:", address)

Request URL: https://nominatim.openstreetmap.org/reverse?osm_type=W&osm_id=123456789&format=json
Error: Received status code 400
Response content: {"error":{"code":400,"message":"Parameter 'lon' missing."}}
Address: None
